In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
s = tf.io.read_file('day11.txt')
s = tf.strings.split(s, '\n')
s = tf.strings.unicode_decode(s, 'UTF-8')
s = s.to_tensor()
# 46 is the ., meaning floor
s -= 46
# Divide by thirty, this means 1 is empty seat, 0 is floor
s /= 30
s = tf.cast(s, tf.float32)

# The not floor mask is 0 on floor tiles, and 1 on others. This makes it easy to fix after convolving
not_floor_mask = s
empty_seats = s
occupied_seats = tf.zeros_like(empty_seats)

In [3]:
@tf.function(experimental_compile=True)
def solve(not_floor_mask):
    kernel = tf.constant([
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1]
    ], tf.float32)[:,:,None,None]

    # Batch and channel dimensions, necessary for conv2D
    not_floor_mask = not_floor_mask[None,:,:,None]
    occupied_seats = tf.zeros_like(not_floor_mask)

    last_occupied = tf.ones_like(occupied_seats)

    while not tf.reduce_all(last_occupied == occupied_seats):
        last_occupied = occupied_seats
        convolution = tf.nn.conv2d(occupied_seats, kernel, strides=1, padding='SAME')

        empty_to_occupied = tf.where(convolution == 0, 1.0, 0.0)
        empty_to_occupied *= (1-occupied_seats)*not_floor_mask

        occupied_to_empty = tf.where(convolution >= 4, 1.0, 0.0)
        occupied_to_empty *= occupied_seats
        
        occupied_seats += (empty_to_occupied - occupied_to_empty)

    return tf.reduce_sum(occupied_seats)

%timeit solve(not_floor_mask)
solve(not_floor_mask)

4.3 ms ± 95.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=2468.0>

In [4]:
n = (occupied_seats.shape[0]//2)
rnge = 2**tf.range(n-1, -1, -1, dtype=tf.float64)

kernel_e = tf.concat((
    tf.zeros((n,2*n+1), tf.float64),
    tf.concat((tf.zeros(n, tf.float64), [0], rnge), 0)[None,:],
    tf.zeros((n,2*n+1), tf.float64),
), axis=0)

kernel_w = tf.concat((
    tf.zeros((n,2*n+1), tf.float64),
    tf.concat((rnge[::-1], [0], tf.zeros(n, tf.float64)), 0)[None,:],
    tf.zeros((n,2*n+1), tf.float64),
), axis=0)

kernel_s = tf.concat((
    tf.zeros((2*n+1,n), tf.float64),
    tf.concat((tf.zeros(n, tf.float64), [0], rnge), 0)[:,None],
    tf.zeros((2*n+1,n), tf.float64),
), axis=1)

kernel_n = tf.concat((
    tf.zeros((2*n+1,n), tf.float64),
    tf.concat((rnge[::-1], [0], tf.zeros(n, tf.float64)), 0)[:,None],
    tf.zeros((2*n+1,n), tf.float64),
), axis=1)

kernel_ne = tf.concat((
    tf.zeros((n, n+1), tf.float64),
    tf.linalg.diag(rnge)[::-1]
), axis=1)
kernel_ne = tf.concat((
    kernel_ne,
    tf.zeros((n+1, 2*n+1), tf.float64)
), axis=0)

kernel_se = tf.concat((
    tf.zeros((n, n+1), tf.float64),
    tf.linalg.diag(rnge)
), axis=1)
kernel_se = tf.concat((
    tf.zeros((n+1, 2*n+1), tf.float64),
    kernel_se,
), axis=0)

kernel_sw = tf.concat((
    tf.linalg.diag(rnge)[:,::-1],
    tf.zeros((n, n+1), tf.float64),
), axis=1)
kernel_sw = tf.concat((
    tf.zeros((n+1, 2*n+1), tf.float64),
    kernel_sw,
), axis=0)

kernel_nw = tf.concat((
    tf.linalg.diag(rnge)[::-1,::-1],
    tf.zeros((n, n+1), tf.float64),
), axis=1)
kernel_nw = tf.concat((
    kernel_nw,
    tf.zeros((n+1, 2*n+1), tf.float64),
), axis=0)

kernels = tf.stack([
    kernel_n,
    kernel_e,
    kernel_s,
    kernel_w,
    kernel_ne,
    kernel_se, 
    kernel_sw,
    kernel_nw
],  axis=-1)[:,:,None,:]

In [5]:
@tf.function(experimental_compile=True)
def solve2(not_floor_mask, kernels):
    # Batch and channel dimensions, necessary for conv2D
    occupied_seats = tf.zeros_like(not_floor_mask, tf.float64)[None,:,:,None]
    not_floor_mask = tf.cast(not_floor_mask[None,:,:,None], tf.float64)

    last_occupied = tf.ones_like(occupied_seats)
    
    while not tf.reduce_all(last_occupied == occupied_seats):
        last_occupied = occupied_seats
        empty_seats = (1 - occupied_seats)*not_floor_mask
        convolution = tf.nn.conv2d(tf.concat((occupied_seats, empty_seats), axis=0), kernels, strides=1, padding='SAME')    

        # If conv[0] > conv[1] in a filter, then an occupied seat is closer than an empty seat
        # conv[0,:,:,2] >= conv[1,:,:,2]
        # So we use math.count_nonzero to see if 
        convolution = tf.math.count_nonzero(convolution[0] > convolution[1], axis=-1, dtype=tf.float64)[None,:,:,None]
        empty_to_occupied = tf.where(convolution == 0.0, tf.constant(1.0, tf.float64), tf.constant(0.0, tf.float64))
        empty_to_occupied *= (1-occupied_seats)*not_floor_mask

        occupied_to_empty = tf.where(convolution >= 5.0, tf.constant(1.0, tf.float64), tf.constant(0.0, tf.float64))
        occupied_to_empty *= occupied_seats
        
        occupied_seats += (empty_to_occupied - occupied_to_empty)

    return tf.reduce_sum(occupied_seats)

%timeit solve2(not_floor_mask, kernels)
solve2(not_floor_mask, kernels)

1.22 s ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float64, numpy=2214.0>